In [6]:
import os

def read_experiment_data(folder_path):
    # Desired order of experiments
    experiment_order = [
        'direct', 'exp6', 'exp61', 'exp52', 'exp53', 'exp4.4', 'exp76'
    ]

    # Desired order of LMs
    lm_order = ['phi3', 'aya', 'command-r']

    # Initialize experiment_data with all values set to "-1±-1\%"
    experiment_data = {exp: {lm: "-1±-1\%" for lm in lm_order} for exp in experiment_order}

    # Mapping for experiment names
    experiment_name_mapping = {
        'direct': 'direct asking',
        'exp6': 'CoT',
        'exp61': 'PS prompting',
        'exp4': '1 inst. 2 messages user dep.',
        'exp54': '1 inst. info. copied user dep.',
        'exp4.4': '1 inst. 2 messages',
        'exp52': '2 inst.',
        'exp5': '2 inst. user dep.',
        'exp7': '1 inst. 1 prompt user dep.',
        'exp76': '1 inst. 1 prompt',
        'exp53': '1 inst. info. copied'
    }

    # Read each file in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            with open(os.path.join(folder_path, file_name), 'r', encoding='utf-8') as file:
                lines = file.readlines()
                # Extract experiment name and language model name from the first line
                first_line_parts = lines[0].strip().split()
                experiment_name = next((part.replace(":", "") for part in first_line_parts if part.startswith('exp') or part == 'direct'), None)
                language_model = first_line_parts[-1].split(':')[0]
                
                # Extract accuracy and SD from the file
                accuracy = None
                sd = None
                for line in lines:
                    if line.startswith('accuracy:'):
                        accuracy = float(line.split(':')[1].strip())
                    if line.startswith('SD:'):
                        sd = float(line.split(':')[1].strip())
                
                # Format accuracy and SD
                if accuracy is not None and sd is not None and experiment_name in experiment_data and language_model in lm_order:
                    accuracy_percent = f"{accuracy * 100:.2f}".rstrip('0').rstrip('.')
                    sd_percent = f"{sd * 100:.1f}".rstrip('0').rstrip('.') + "\\%"
                    result = f"{accuracy_percent}±{sd_percent}"
                
                    # Store the formatted result in the dictionary
                    experiment_data[experiment_name][language_model] = result

    # Calculate averages for each experiment
    experiment_averages = {
        exp: sum(float(experiment_data[exp][lm].split('±')[0]) for lm in lm_order) / len(lm_order)
        for exp in experiment_data
    }

    # Get the average for "CoT"
    CoT_average = experiment_averages.get('exp6', None)

    # Generate LaTeX table
    table_header = "Experiment & Phi-3 & Aya & Command-R & Avg. Diff \\\\"

    table_rows = []
    for exp in experiment_order:
        if exp in experiment_data:
            mapped_experiment_name = experiment_name_mapping.get(exp, exp)
            row = mapped_experiment_name
            for lm in lm_order:
                value = experiment_data[exp].get(lm)
                accuracy_value = float(value.split('±')[0])
                
                # Check if value is the highest in its column
                if accuracy_value == max(float(experiment_data[exp][lm].split('±')[0]) for exp in experiment_order if lm in experiment_data[exp]):
                    row += (" & \\textbf" + '{' + f"{value}" + "}")
                else:
                    row += f" & {value}"

            # Calculate and format the average difference
            avg_diff = experiment_averages[exp] - CoT_average
            if avg_diff > 0:
                avg_diff_str = f"\\textcolor{{blue}}{{{avg_diff:.2f}}}".rstrip('0').rstrip('.')
            else:
                avg_diff_str = f"\\textcolor{{red}}{{{avg_diff:.2f}}}".rstrip('0').rstrip('.')

            row += f" & {avg_diff_str}"
            table_rows.append(row)

    table_body = " \\\\\n".join(table_rows)
    latex_table = (
        "\\begin{table}[h!]\n"
        "\\caption{Our Dataset}\n"
        "\\begin{tabular}{l" + "c" * (len(lm_order) + 1) + "}\n"
        "\\hline\n"
        + table_header + "\n\\hline\n"
        + table_body + " \\\\" + "\n\\hline\n"
        "\\end{tabular}\n"
        "\\end{table}"
    )

    return latex_table


# Example usage
folder_path = 'i'
latex_code = read_experiment_data(folder_path)
print(latex_code)


\begin{table}[h!]
\caption{Ours Dataset}
\begin{tabular}{lcccc}
\hline
Experiment & Phi-3 & Aya & Command-R & Avg. Diff \\
\hline
direct asking & 67±4.7\% & 55±5\% & 54±5\% & \textcolor{red}{-6.67} \\
CoT & \textbf{78±4.1\%} & 59±4.9\% & 59±4.9\% & \textcolor{red}{0.00} \\
PS prompting & 72±4.5\% & 63±4.8\% & \textbf{66±4.7\%} & \textcolor{blue}{1.67} \\
2 inst. & 70±4.6\% & 67±4.7\% & \textbf{66±4.7\%} & \textcolor{blue}{2.33} \\
1 inst. info. copied & 70±4.6\% & 64±4.8\% & 58±4.9\% & \textcolor{red}{-1.33} \\
1 inst. 2 messages & 71±4.5\% & 62±4.9\% & 46±5\% & \textcolor{red}{-5.67} \\
1 inst. 1 prompt & 73±4.4\% & \textbf{68±4.7\%} & 56±5\% & \textcolor{blue}{0.33} \\
\hline
\end{tabular}
\end{table}


In [7]:
import os

def read_experiment_data(folder_path):
    # Desired order of experiments
    experiment_order = [
        'direct', 'exp6', 'exp61', 'exp5', 'exp54', 'exp4',  'exp7'
    ]

    # Desired order of LMs
    lm_order = ['phi3', 'aya', 'command-r']

    # Initialize experiment_data with all values set to "-1±-1\%"
    experiment_data = {exp: {lm: "-1±-1\%" for lm in lm_order} for exp in experiment_order}

    # Mapping for experiment names
    experiment_name_mapping = {
        'direct': 'direct asking',
        'exp6': 'CoT',
        'exp61': 'PS prompting',
        'exp4': '1 inst. 2 messages user dep.',
        'exp54': '1 inst. info. copied user dep.',
        'exp4.4': '1 inst. 2 messages',
        'exp52': '2 inst.',
        'exp5': '2 inst. user dep.',
        'exp7': '1 inst. 1 prompt user dep.',
        'exp76': '1 inst. 1 prompt',
        'exp53': '1 inst. info. copied'
    }

    # Read each file in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            with open(os.path.join(folder_path, file_name), 'r', encoding='utf-8') as file:
                lines = file.readlines()
                # Extract experiment name and language model name from the first line
                first_line_parts = lines[0].strip().split()
                experiment_name = next((part.replace(":", "") for part in first_line_parts if part.startswith('exp') or part == 'direct'), None)
                language_model = first_line_parts[-1].split(':')[0]
                
                # Extract accuracy and SD from the file
                accuracy = None
                sd = None
                for line in lines:
                    if line.startswith('accuracy:'):
                        accuracy = float(line.split(':')[1].strip())
                    if line.startswith('SD:'):
                        sd = float(line.split(':')[1].strip())
                
                # Format accuracy and SD
                if accuracy is not None and sd is not None and experiment_name in experiment_data and language_model in lm_order:
                    accuracy_percent = f"{accuracy * 100:.2f}".rstrip('0').rstrip('.')
                    sd_percent = f"{sd * 100:.1f}".rstrip('0').rstrip('.') + "\\%"
                    result = f"{accuracy_percent}±{sd_percent}"
                
                    # Store the formatted result in the dictionary
                    experiment_data[experiment_name][language_model] = result

    # Calculate averages for each experiment
    experiment_averages = {
        exp: sum(float(experiment_data[exp][lm].split('±')[0]) for lm in lm_order) / len(lm_order)
        for exp in experiment_data
    }

    # Get the average for "CoT"
    CoT_average = experiment_averages.get('exp6', None)

    # Generate LaTeX table
    table_header = "Experiment & Phi-3 & Aya & Command-R & Avg. Diff \\\\"

    table_rows = []
    for exp in experiment_order:
        if exp in experiment_data:
            mapped_experiment_name = experiment_name_mapping.get(exp, exp)
            row = mapped_experiment_name
            for lm in lm_order:
                value = experiment_data[exp].get(lm)
                accuracy_value = float(value.split('±')[0])
                
                # Check if value is the highest in its column
                if accuracy_value == max(float(experiment_data[exp][lm].split('±')[0]) for exp in experiment_order if lm in experiment_data[exp]):
                    row += (" & \\textbf" + '{' + f"{value}" + "}")
                else:
                    row += f" & {value}"

            # Calculate and format the average difference
            avg_diff = experiment_averages[exp] - CoT_average
            if avg_diff > 0:
                avg_diff_str = f"\\textcolor{{blue}}{{{avg_diff:.2f}}}".rstrip('0').rstrip('.')
            else:
                avg_diff_str = f"\\textcolor{{red}}{{{avg_diff:.2f}}}".rstrip('0').rstrip('.')

            row += f" & {avg_diff_str}"
            table_rows.append(row)

    table_body = " \\\\\n".join(table_rows)
    latex_table = (
        "\\begin{table}[h!]\n"
        "\\caption{Our Dataset, User-Knowledge Dependent Prompts}\n"
        "\\begin{tabular}{l" + "c" * (len(lm_order) + 1) + "}\n"
        "\\hline\n"
        + table_header + "\n\\hline\n"
        + table_body + " \\\\" + "\n\\hline\n"
        "\\end{tabular}\n"
        "\\end{table}"
    )

    return latex_table


# Example usage
folder_path = 'dep'
latex_code = read_experiment_data(folder_path)
print(latex_code)


\begin{table}[h!]
\caption{Our Dataset, User-Knowledge Dependent Prompts}
\begin{tabular}{lcccc}
\hline
Experiment & Phi-3 & Aya & Command-R & Avg. Diff \\
\hline
direct asking & 67±4.7\% & 55±5\% & 54±5\% & \textcolor{red}{-6.67} \\
CoT & \textbf{78±4.1\%} & 59±4.9\% & 59±4.9\% & \textcolor{red}{0.00} \\
PS prompting & 72±4.5\% & 63±4.8\% & 66±4.7\% & \textcolor{blue}{1.67} \\
2 inst. user dep. & 62±4.9\% & \textbf{71±4.5\%} & \textbf{74±4.4\%} & \textcolor{blue}{3.67} \\
1 inst. info. copied user dep. & 61±4.9\% & 68±4.7\% & 60±4.9\% & \textcolor{red}{-2.33} \\
1 inst. 2 messages user dep. & 60±4.9\% & 60±4.9\% & 62±4.9\% & \textcolor{red}{-4.67} \\
1 inst. 1 prompt user dep. & 65±4.8\% & 58±4.9\% & 64±4.8\% & \textcolor{red}{-3.00} \\
\hline
\end{tabular}
\end{table}
